# Overview

In this notebook, I conduct the design and development of artificial intelligence models. The main goal of it is to try to predict the physical exercise adherence of the users based on how much time they have been exercising. 

**Author**: Jon Maestre Escobar

**Email**: jonmaestre@opendeusto.es.

In [1]:
import pandas as pd
import numpy as np
import warnings
import pandas as pd
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import time
from utilities import Data_cleaning

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale

import math
import copy
import re
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
df_filtered_merged = pd.read_hdf('../data/filtered_merged_dataset_v1.h5', key='df')
df_filtered_merged.fillna(0, inplace=True)
df_filtered_merged.shape

(46020, 8705)

## **Users Filtered by Different Training Periods**

In this section, I have undertaken a detailed process to filter and analyze users based on the duration of their training periods recorded in our dataset. My objective was to categorize users into different groups based on their training consistency and ensure comprehensive data analysis by focusing on the range of days from the first to the last recorded training session.

**Steps Taken**

- **Initial Filtering**:
    - **30 Days or Less**: I first identified users whose training period spans 30 days or less from their first recorded training session to their last.
    - **31 to 90 Days**: Next, I filtered users whose training period spans more than 30 days but not exceeding 90 days.
    - **91 to 180 Days**: Similarly, I filtered users whose training period spans more than 90 days but up to 180 days.
    - **181 to 365 Days**: Finally, I identified users whose training period spans more than 180 days but not exceeding 365 days.

- **Data Completion**:
    - For each of these groups, I ensured that the training records were complete. Specifically, I filled in any missing days with zeros from the user's first recorded training day to ensure there are no gaps in the data.
    - This involved generating a complete date range for each user based on their first training session and merging it with the existing records, filling in the missing entries.

- **Detailed Analysis**:
    - The filtered and completed datasets were then analyzed to understand user behavior and training patterns better. This step helps in identifying trends and making data-driven recommendations for improving user engagement and training adherence.

By implementing these steps, I aim to provide a robust analysis framework that accurately reflects users' training habits, allowing us to draw meaningful insights and support the development of personalized fitness interventions. This comprehensive approach ensures that our analysis accounts for all training days, even those that were not initially recorded, providing a more accurate picture of user activity.

In [3]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is 30 days or less
users_30_days_or_less = user_training_period[user_training_period['training_period_days'] <= 30]

# Create a DataFrame with the filtered users
df_filtered_30days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_days_or_less['user_programs_user_id'])]
df_filtered_30days.shape

(4596, 8705)

In [4]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 30 and 90 days
users_30_to_90_days = user_training_period[(user_training_period['training_period_days'] > 30) & (user_training_period['training_period_days'] <= 90)]

# Create a DataFrame with the filtered users
df_filtered_30_to_90_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_to_90_days['user_programs_user_id'])]
df_filtered_30_to_90_days.shape

(7006, 8705)

In [5]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 90 and 180 days
users_90_to_180_days = user_training_period[(user_training_period['training_period_days'] > 90) & (user_training_period['training_period_days'] <= 180)]

# Create a DataFrame with the filtered users
df_filtered_90_to_180_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_90_to_180_days['user_programs_user_id'])]
df_filtered_90_to_180_days.shape

(13243, 8705)

In [6]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 180 and 365 days
users_180_to_365_days = user_training_period[(user_training_period['training_period_days'] > 180) & (user_training_period['training_period_days'] <= 365)]

# Create a DataFrame with the filtered users
df_filtered_180_to_365_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_180_to_365_days['user_programs_user_id'])]
df_filtered_180_to_365_days.shape

(21175, 8705)

In [7]:
print(f'The number of user who have trained 30 days or less is:', df_filtered_30days.user_programs_user_id.nunique()) 
print(f'The number of user who have trained between 31 and 90 days is:', df_filtered_30_to_90_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 91 and 180 days is', df_filtered_90_to_180_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 181 and 365 days is', df_filtered_180_to_365_days.user_programs_user_id.nunique())

The number of user who have trained 30 days or less is: 1658
The number of user who have trained between 31 and 90 days is: 549
The number of user who have trained between 91 and 180 days is 572
The number of user who have trained between 181 and 365 days is 409


### **Complete With Zeros Non-Training Days**

In this part of the analysis, I focused on ensuring that each user's training record is complete by filling in non-training days with zeros. This step is crucial for accurate time series analysis and to avoid any misinterpretation of the user's training consistency. Here are the detailed steps I followed:

- **Identify First and Last Training Day**:
    - For each user in the filtered datasets, I identified the first and last day they recorded a training session. This serves as the basis for generating a complete date range for each user.

- **Generate Complete Date Ranges**:
    - Using the first training day as the starting point, I generated a complete date range for each user up to the specified number of days (30 days for users with up to 30 days of training, 90 days for users with training periods of 31-90 days, 180 days for users with training periods of 91-180 days, and 365 days for users with training periods of 181-365 days).

- **Merge with Original Data and Fill Missing Days**:
    - I merged these generated date ranges with the original user training data to identify days that were not recorded.
    - For the missing days, I filled in the relevant columns with zeros, ensuring that there are no gaps in the training data.

- **Combine Completed Data**:
    - The completed data for each user was then combined into a single DataFrame. This new DataFrame includes all the original training data as well as the newly added rows for the non-training days filled with zeros.

This process ensures that each user has a continuous record of training activity, allowing for more accurate analysis and modeling. By filling in the non-training days with zeros, I can better understand user behavior and training patterns, which is essential for developing effective AI models and making informed recommendations.

In [12]:
# Ensure that 'date' is in datetime format
df_filtered_30days['date'] = pd.to_datetime(df_filtered_30days['date'])

# Get the first training day for each user
first_training_day = df_filtered_30days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day['user_programs_user_id'], first_training_day['first_training_date']):
    # Generate a date range from the first day up to 30 days later
    date_range = pd.date_range(start=first_day, periods=30)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30days[df_filtered_30days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30days = pd.concat([completed_data_30days, user_data])

# Reset the index of the final DataFrame
completed_data_30days.reset_index(drop=True, inplace=True)
completed_data_30days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [13]:
completed_data_30days.to_hdf('../data/completed_data_30days_v1.h5', key='df')

In [3]:
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
print(f'30 days dataset shape:', completed_data_30days.shape)

30 days dataset shape: (49740, 8705)


In [15]:
# Ensure that 'date' is in datetime format
df_filtered_30_to_90_days['date'] = pd.to_datetime(df_filtered_30_to_90_days['date'])

# Get the first training day for each user
first_training_day_30_to_90 = df_filtered_30_to_90_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_30_to_90.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30_to_90_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_30_to_90['user_programs_user_id'], first_training_day_30_to_90['first_training_date']):
    # Generate a date range from the first day up to 90 days later
    date_range = pd.date_range(start=first_day, periods=90)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30_to_90_days[df_filtered_30_to_90_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30_to_90_days = pd.concat([completed_data_30_to_90_days, user_data])

# Reset the index of the final DataFrame
completed_data_30_to_90_days.reset_index(drop=True, inplace=True)
completed_data_30_to_90_days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [16]:
completed_data_30_to_90_days.to_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')

In [9]:
completed_data_30_to_90_days = pd.read_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
print(f'30-90 days dataset shape:', completed_data_30_to_90_days.shape)

30-90 days dataset shape: (49410, 8705)


In [8]:
# Ensure that 'date' is in datetime format
df_filtered_90_to_180_days['date'] = pd.to_datetime(df_filtered_90_to_180_days['date'])

# Get the first training day for each user
first_training_day_90_to_180 = df_filtered_90_to_180_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_90_to_180.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_90_to_180_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_90_to_180['user_programs_user_id'], first_training_day_90_to_180['first_training_date']):
    # Generate a date range from the first day up to 180 days later
    date_range = pd.date_range(start=first_day, periods=180)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_90_to_180_days[df_filtered_90_to_180_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_90_to_180_days = pd.concat([completed_data_90_to_180_days, user_data])

# Reset the index of the final DataFrame
completed_data_90_to_180_days.reset_index(drop=True, inplace=True)
completed_data_90_to_180_days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [ ]:
completed_data_90_to_180_days.to_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')

In [10]:
completed_data_90_to_180_days = pd.read_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
print(f'90-180 days dataset shape:', completed_data_90_to_180_days.shape)

90-180 days dataset shape: (102960, 8705)


In [30]:
# Ensure that 'date' is in datetime format
df_filtered_180_to_365_days['date'] = pd.to_datetime(df_filtered_180_to_365_days['date'])

# Get the first training day for each user
first_training_day_180_to_365 = df_filtered_180_to_365_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_180_to_365.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_180_to_365_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_180_to_365['user_programs_user_id'], first_training_day_180_to_365['first_training_date']):
    # Generate a date range from the first day up to 365 days later
    date_range = pd.date_range(start=first_day, periods=365)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_180_to_365_days[df_filtered_180_to_365_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_180_to_365_days = pd.concat([completed_data_180_to_365_days, user_data])

# Reset the index of the final DataFrame
completed_data_180_to_365_days.reset_index(drop=True, inplace=True)
print(completed_data_180_to_365_days)

        user_programs_user_id       date session_executions_updated_at  \
0                         108 2021-06-11    2021-06-11 18:00:35.640406   
1                         108 2021-06-12                             0   
2                         108 2021-06-13                             0   
3                         108 2021-06-14                             0   
4                         108 2021-06-15                             0   
...                       ...        ...                           ...   
149280                  11835 2022-11-22                             0   
149281                  11835 2022-11-23                             0   
149282                  11835 2022-11-24                             0   
149283                  11835 2022-11-25                             0   
149284                  11835 2022-11-26                             0   

        1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                              0.0          

In [ ]:
# Memory error
completed_data_180_to_365_days.to_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

In [42]:
# Save the DataFrames to a CSV file
completed_data_30days.to_csv('../data/completed_data_30_days.csv', index=False)
completed_data_30_to_90_days.to_csv('../data/completed_data_30_to_90_days.csv', index=False)
completed_data_90_to_180_days.to_csv('../data/completed_data_90_to_180_days.csv', index=False)
completed_data_180_to_365_days.to_csv('../data/completed_data_180_to_365_days.csv', index=False)

In [ ]:
# TODAVIA NO ESTA HECHO!!
# Falta el dataset de 180-365 dias!!

# Read the datasets
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
completed_data_30_to_90_days = pd.read_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
completed_data_90_to_180_days = pd.read_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
completed_data_180_to_365_days = pd.read_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

print(f'30 days dataset shape:', completed_data_30days.shape)
print(f'30-90 days dataset shape:', completed_data_30_to_90_days.shape)
print(f'90-180 days dataset shape:', completed_data_90_to_180_days.shape)
print(f'180-365 days dataset shape:', completed_data_180_to_365_days.shape)

## **Time Series to Supervised Learning**

In [2]:
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
print(f'30 days dataset shape:', completed_data_30days.shape)

30 days dataset shape: (49740, 8705)


In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # Operation 1
    start_time = time.time()
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    print("--- %s seconds ---" % (time.time() - start_time))

    # Operation 2
    # input sequence (t-n, ... t-1)
    start_time = time.time()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    print("--- %s seconds ---" % (time.time() - start_time))


    # Operation 3
    start_time = time.time()
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    print("--- %s seconds ---" % (time.time() - start_time))

    #Operation 4
    # put it all together
    start_time = time.time()
    agg = concat(cols, axis=1)
    agg.columns = names
    print("--- %s seconds ---" % (time.time() - start_time))

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    del start_time, n_vars, df, cols, names, data, n_in, n_out, dropnan, i

    return agg 

In [4]:
df_test_30days_part1 = completed_data_30days[0:10000]

In [5]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part1 = series_to_supervised(df_test_30days_part1.values, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part1.head()

--- 2.2541682720184326 seconds ---
--- 5.695459604263306 seconds ---
--- 2.427774667739868 seconds ---
--- 8.046605110168457 seconds ---


,var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var6(t-7),var7(t-7),var8(t-7),var9(t-7),var10(t-7),var11(t-7),var12(t-7),var13(t-7),var14(t-7),var15(t-7),var16(t-7),var17(t-7),var18(t-7),var19(t-7),var20(t-7),var21(t-7),var22(t-7),var23(t-7),var24(t-7),var25(t-7),var26(t-7),var27(t-7),var28(t-7),var29(t-7),var30(t-7),var31(t-7),var32(t-7),var33(t-7),var34(t-7),var35(t-7),var36(t-7),var37(t-7),var38(t-7),var39(t-7),var40(t-7),var41(t-7),var42(t-7),var43(t-7),var44(t-7),var45(t-7),var46(t-7),var47(t-7),var48(t-7),var49(t-7),var50(t-7),var51(t-7),var52(t-7),var53(t-7),var54(t-7),var55(t-7),var56(t-7),var57(t-7),var58(t-7),var59(t-7),var60(t-7),var61(t-7),var62(t-7),var63(t-7),var64(t-7),var65(t-7),var66(t-7),var67(t-7),var68(t-7),var69(t-7),var70(t-7),var71(t-7),var72(t-7),var73(t-7),var74(t-7),var75(t-7),var76(t-7),var77(t-7),var78(t-7),var79(t-7),var80(t-7),var81(t-7),var82(t-7),var83(t-7),var84(t-7),var85(t-7),var86(t-7),var87(t-7),var88(t-7),var89(t-7),var90(t-7),var91(t-7),var92(t-7),var93(t-7),var94(t-7),var95(t-7),var96(t-7),var97(t-7),var98(t-7),var99(t-7),var100(t-7),var101(t-7),var102(t-7),var103(t-7),var104(t-7),var105(t-7),var106(t-7),var107(t-7),var108(t-7),var109(t-7),var110(t-7),var111(t-7),var112(t-7),var113(t-7),var114(t-7),var115(t-7),var116(t-7),var117(t-7),var118(t-7),var119(t-7),var120(t-7),var121(t-7),var122(t-7),var123(t-7),var124(t-7),var125(t-7),var126(t-7),var127(t-7),var128(t-7),var129(t-7),var130(t-7),var131(t-7),var132(t-7),var133(t-7),var134(t-7),var135(t-7),var136(t-7),var137(t-7),var138(t-7),var139(t-7),var140(t-7),var141(t-7),var142(t-7),var143(t-7),var144(t-7),var145(t-7),var146(t-7),var147(t-7),var148(t-7),var149(t-7),var150(t-7),var151(t-7),var152(t-7),var153(t-7),var154(t-7),var155(t-7),var156(t-7),var157(t-7),var158(t-7),var159(t-7),var160(t-7),var161(t-7),var162(t-7),var163(t-7),var164(t-7),var165(t-7),var166(t-7),var167(t-7),var168(t-7),var169(t-7),var170(t-7),var171(t-7),var172(t-7),var173(t-7),var174(t-7),var175(t-7),var176(t-7),var177(t-7),var178(t-7),var179(t-7),var180(t-7),var181(t-7),var182(t-7),var183(t-7),var184(t-7),var185(t-7),var186(t-7),var187(t-7),var188(t-7),var189(t-7),var190(t-7),var191(t-7),var192(t-7),var193(t-7),var194(t-7),var195(t-7),var196(t-7),var197(t-7),var198(t-7),var199(t-7),var200(t-7),var201(t-7),var202(t-7),var203(t-7),var204(t-7),var205(t-7),var206(t-7),var207(t-7),var208(t-7),var209(t-7),var210(t-7),var211(t-7),var212(t-7),var213(t-7),var214(t-7),var215(t-7),var216(t-7),var217(t-7),var218(t-7),var219(t-7),var220(t-7),var221(t-7),var222(t-7),var223(t-7),var224(t-7),var225(t-7),var226(t-7),var227(t-7),var228(t-7),var229(t-7),var230(t-7),var231(t-7),var232(t-7),var233(t-7),var234(t-7),var235(t-7),var236(t-7),var237(t-7),var238(t-7),var239(t-7),var240(t-7),var241(t-7),var242(t-7),var243(t-7),var244(t-7),var245(t-7),var246(t-7),var247(t-7),var248(t-7),var249(t-7),var250(t-7),...,var8456(t+2),var8457(t+2),var8458(t+2),var8459(t+2),var8460(t+2),var8461(t+2),var8462(t+2),var8463(t+2),var8464(t+2),var8465(t+2),var8466(t+2),var8467(t+2),var8468(t+2),var8469(t+2),var8470(t+2),var8471(t+2),var8472(t+2),var8473(t+2),var8474(t+2),var8475(t+2),var8476(t+2),var8477(t+2),var8478(t+2),var8479(t+2),var8480(t+2),var8481(t+2),var8482(t+2),var8483(t+2),var8484(t+2),var8485(t+2),var8486(t+2),var8487(t+2),var8488(t+2),var8489(t+2),var8490(t+2),var8491(t+2),var8492(t+2),var8493(t+2),var8494(t+2),var8495(t+2),var8496(t+2),var8497(t+2),var8498(t+2),var8499(t+2),var8500(t+2),var8501(t+2),var8502(t+2),var8503(t+2),var8504(t+2),var8505(t+2),var8506(t+2),var8507(t+2),var8508(t+2),var8509(t+2),var8510(t+2),var8511(t+2),var8512(t+2),var8513(t+2),var8514(t+2),var8515(t+2),var8516(t+2),var8517(t+2),var8518(t+2),var8519(t+2),var8520(t+2),var8521(t+2),var8522(t+2),var8523(t+2),var8524(t+2),var8525(t+2),var8526(t+2),var8527(t+2),var8528(t+2),var8529(t+2),var8530(t+2),var8531(t+2),var8532(t+2),var8533(t+2),var8534(t+2),var8535(t+2),var8536(t+2),var8537(t+2),var8538(t+2),var8539(t+2),var8540(t+2

In [6]:
# Save the DataFrame to a CSV file
reframed_data_30days_part1.to_csv('../data/reframed_data_30days_part1.csv', index=False)
reframed_data_30days_part1.shape

(9991, 87050)

In [4]:
df_test_30days_part2 = completed_data_30days[10000:20000]

In [5]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part2 = series_to_supervised(df_test_30days_part2.values, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part2.head()

--- 2.2674264907836914 seconds ---
--- 5.7294347286224365 seconds ---
--- 2.436472177505493 seconds ---
--- 8.083970785140991 seconds ---


,var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var6(t-7),var7(t-7),var8(t-7),var9(t-7),var10(t-7),var11(t-7),var12(t-7),var13(t-7),var14(t-7),var15(t-7),var16(t-7),var17(t-7),var18(t-7),var19(t-7),var20(t-7),var21(t-7),var22(t-7),var23(t-7),var24(t-7),var25(t-7),var26(t-7),var27(t-7),var28(t-7),var29(t-7),var30(t-7),var31(t-7),var32(t-7),var33(t-7),var34(t-7),var35(t-7),var36(t-7),var37(t-7),var38(t-7),var39(t-7),var40(t-7),var41(t-7),var42(t-7),var43(t-7),var44(t-7),var45(t-7),var46(t-7),var47(t-7),var48(t-7),var49(t-7),var50(t-7),var51(t-7),var52(t-7),var53(t-7),var54(t-7),var55(t-7),var56(t-7),var57(t-7),var58(t-7),var59(t-7),var60(t-7),var61(t-7),var62(t-7),var63(t-7),var64(t-7),var65(t-7),var66(t-7),var67(t-7),var68(t-7),var69(t-7),var70(t-7),var71(t-7),var72(t-7),var73(t-7),var74(t-7),var75(t-7),var76(t-7),var77(t-7),var78(t-7),var79(t-7),var80(t-7),var81(t-7),var82(t-7),var83(t-7),var84(t-7),var85(t-7),var86(t-7),var87(t-7),var88(t-7),var89(t-7),var90(t-7),var91(t-7),var92(t-7),var93(t-7),var94(t-7),var95(t-7),var96(t-7),var97(t-7),var98(t-7),var99(t-7),var100(t-7),var101(t-7),var102(t-7),var103(t-7),var104(t-7),var105(t-7),var106(t-7),var107(t-7),var108(t-7),var109(t-7),var110(t-7),var111(t-7),var112(t-7),var113(t-7),var114(t-7),var115(t-7),var116(t-7),var117(t-7),var118(t-7),var119(t-7),var120(t-7),var121(t-7),var122(t-7),var123(t-7),var124(t-7),var125(t-7),var126(t-7),var127(t-7),var128(t-7),var129(t-7),var130(t-7),var131(t-7),var132(t-7),var133(t-7),var134(t-7),var135(t-7),var136(t-7),var137(t-7),var138(t-7),var139(t-7),var140(t-7),var141(t-7),var142(t-7),var143(t-7),var144(t-7),var145(t-7),var146(t-7),var147(t-7),var148(t-7),var149(t-7),var150(t-7),var151(t-7),var152(t-7),var153(t-7),var154(t-7),var155(t-7),var156(t-7),var157(t-7),var158(t-7),var159(t-7),var160(t-7),var161(t-7),var162(t-7),var163(t-7),var164(t-7),var165(t-7),var166(t-7),var167(t-7),var168(t-7),var169(t-7),var170(t-7),var171(t-7),var172(t-7),var173(t-7),var174(t-7),var175(t-7),var176(t-7),var177(t-7),var178(t-7),var179(t-7),var180(t-7),var181(t-7),var182(t-7),var183(t-7),var184(t-7),var185(t-7),var186(t-7),var187(t-7),var188(t-7),var189(t-7),var190(t-7),var191(t-7),var192(t-7),var193(t-7),var194(t-7),var195(t-7),var196(t-7),var197(t-7),var198(t-7),var199(t-7),var200(t-7),var201(t-7),var202(t-7),var203(t-7),var204(t-7),var205(t-7),var206(t-7),var207(t-7),var208(t-7),var209(t-7),var210(t-7),var211(t-7),var212(t-7),var213(t-7),var214(t-7),var215(t-7),var216(t-7),var217(t-7),var218(t-7),var219(t-7),var220(t-7),var221(t-7),var222(t-7),var223(t-7),var224(t-7),var225(t-7),var226(t-7),var227(t-7),var228(t-7),var229(t-7),var230(t-7),var231(t-7),var232(t-7),var233(t-7),var234(t-7),var235(t-7),var236(t-7),var237(t-7),var238(t-7),var239(t-7),var240(t-7),var241(t-7),var242(t-7),var243(t-7),var244(t-7),var245(t-7),var246(t-7),var247(t-7),var248(t-7),var249(t-7),var250(t-7),...,var8456(t+2),var8457(t+2),var8458(t+2),var8459(t+2),var8460(t+2),var8461(t+2),var8462(t+2),var8463(t+2),var8464(t+2),var8465(t+2),var8466(t+2),var8467(t+2),var8468(t+2),var8469(t+2),var8470(t+2),var8471(t+2),var8472(t+2),var8473(t+2),var8474(t+2),var8475(t+2),var8476(t+2),var8477(t+2),var8478(t+2),var8479(t+2),var8480(t+2),var8481(t+2),var8482(t+2),var8483(t+2),var8484(t+2),var8485(t+2),var8486(t+2),var8487(t+2),var8488(t+2),var8489(t+2),var8490(t+2),var8491(t+2),var8492(t+2),var8493(t+2),var8494(t+2),var8495(t+2),var8496(t+2),var8497(t+2),var8498(t+2),var8499(t+2),var8500(t+2),var8501(t+2),var8502(t+2),var8503(t+2),var8504(t+2),var8505(t+2),var8506(t+2),var8507(t+2),var8508(t+2),var8509(t+2),var8510(t+2),var8511(t+2),var8512(t+2),var8513(t+2),var8514(t+2),var8515(t+2),var8516(t+2),var8517(t+2),var8518(t+2),var8519(t+2),var8520(t+2),var8521(t+2),var8522(t+2),var8523(t+2),var8524(t+2),var8525(t+2),var8526(t+2),var8527(t+2),var8528(t+2),var8529(t+2),var8530(t+2),var8531(t+2),var8532(t+2),var8533(t+2),var8534(t+2),var8535(t+2),var8536(t+2),var8537(t+2),var8538(t+2),var8539(t+2),var8540(t+2

In [6]:
# Save the DataFrame to a CSV file
reframed_data_30days_part2.to_csv('../data/reframed_data_30days_part2.csv', index=False)
reframed_data_30days_part2.shape

(9991, 87050)

In [4]:
df_test_30days_part3 = completed_data_30days[20000:30000]

In [5]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part3 = series_to_supervised(df_test_30days_part3.values, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part3.head()

--- 2.248019218444824 seconds ---
--- 5.736642360687256 seconds ---
--- 2.4465153217315674 seconds ---
--- 8.103708028793335 seconds ---


,var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var6(t-7),var7(t-7),var8(t-7),var9(t-7),var10(t-7),var11(t-7),var12(t-7),var13(t-7),var14(t-7),var15(t-7),var16(t-7),var17(t-7),var18(t-7),var19(t-7),var20(t-7),var21(t-7),var22(t-7),var23(t-7),var24(t-7),var25(t-7),var26(t-7),var27(t-7),var28(t-7),var29(t-7),var30(t-7),var31(t-7),var32(t-7),var33(t-7),var34(t-7),var35(t-7),var36(t-7),var37(t-7),var38(t-7),var39(t-7),var40(t-7),var41(t-7),var42(t-7),var43(t-7),var44(t-7),var45(t-7),var46(t-7),var47(t-7),var48(t-7),var49(t-7),var50(t-7),var51(t-7),var52(t-7),var53(t-7),var54(t-7),var55(t-7),var56(t-7),var57(t-7),var58(t-7),var59(t-7),var60(t-7),var61(t-7),var62(t-7),var63(t-7),var64(t-7),var65(t-7),var66(t-7),var67(t-7),var68(t-7),var69(t-7),var70(t-7),var71(t-7),var72(t-7),var73(t-7),var74(t-7),var75(t-7),var76(t-7),var77(t-7),var78(t-7),var79(t-7),var80(t-7),var81(t-7),var82(t-7),var83(t-7),var84(t-7),var85(t-7),var86(t-7),var87(t-7),var88(t-7),var89(t-7),var90(t-7),var91(t-7),var92(t-7),var93(t-7),var94(t-7),var95(t-7),var96(t-7),var97(t-7),var98(t-7),var99(t-7),var100(t-7),var101(t-7),var102(t-7),var103(t-7),var104(t-7),var105(t-7),var106(t-7),var107(t-7),var108(t-7),var109(t-7),var110(t-7),var111(t-7),var112(t-7),var113(t-7),var114(t-7),var115(t-7),var116(t-7),var117(t-7),var118(t-7),var119(t-7),var120(t-7),var121(t-7),var122(t-7),var123(t-7),var124(t-7),var125(t-7),var126(t-7),var127(t-7),var128(t-7),var129(t-7),var130(t-7),var131(t-7),var132(t-7),var133(t-7),var134(t-7),var135(t-7),var136(t-7),var137(t-7),var138(t-7),var139(t-7),var140(t-7),var141(t-7),var142(t-7),var143(t-7),var144(t-7),var145(t-7),var146(t-7),var147(t-7),var148(t-7),var149(t-7),var150(t-7),var151(t-7),var152(t-7),var153(t-7),var154(t-7),var155(t-7),var156(t-7),var157(t-7),var158(t-7),var159(t-7),var160(t-7),var161(t-7),var162(t-7),var163(t-7),var164(t-7),var165(t-7),var166(t-7),var167(t-7),var168(t-7),var169(t-7),var170(t-7),var171(t-7),var172(t-7),var173(t-7),var174(t-7),var175(t-7),var176(t-7),var177(t-7),var178(t-7),var179(t-7),var180(t-7),var181(t-7),var182(t-7),var183(t-7),var184(t-7),var185(t-7),var186(t-7),var187(t-7),var188(t-7),var189(t-7),var190(t-7),var191(t-7),var192(t-7),var193(t-7),var194(t-7),var195(t-7),var196(t-7),var197(t-7),var198(t-7),var199(t-7),var200(t-7),var201(t-7),var202(t-7),var203(t-7),var204(t-7),var205(t-7),var206(t-7),var207(t-7),var208(t-7),var209(t-7),var210(t-7),var211(t-7),var212(t-7),var213(t-7),var214(t-7),var215(t-7),var216(t-7),var217(t-7),var218(t-7),var219(t-7),var220(t-7),var221(t-7),var222(t-7),var223(t-7),var224(t-7),var225(t-7),var226(t-7),var227(t-7),var228(t-7),var229(t-7),var230(t-7),var231(t-7),var232(t-7),var233(t-7),var234(t-7),var235(t-7),var236(t-7),var237(t-7),var238(t-7),var239(t-7),var240(t-7),var241(t-7),var242(t-7),var243(t-7),var244(t-7),var245(t-7),var246(t-7),var247(t-7),var248(t-7),var249(t-7),var250(t-7),...,var8456(t+2),var8457(t+2),var8458(t+2),var8459(t+2),var8460(t+2),var8461(t+2),var8462(t+2),var8463(t+2),var8464(t+2),var8465(t+2),var8466(t+2),var8467(t+2),var8468(t+2),var8469(t+2),var8470(t+2),var8471(t+2),var8472(t+2),var8473(t+2),var8474(t+2),var8475(t+2),var8476(t+2),var8477(t+2),var8478(t+2),var8479(t+2),var8480(t+2),var8481(t+2),var8482(t+2),var8483(t+2),var8484(t+2),var8485(t+2),var8486(t+2),var8487(t+2),var8488(t+2),var8489(t+2),var8490(t+2),var8491(t+2),var8492(t+2),var8493(t+2),var8494(t+2),var8495(t+2),var8496(t+2),var8497(t+2),var8498(t+2),var8499(t+2),var8500(t+2),var8501(t+2),var8502(t+2),var8503(t+2),var8504(t+2),var8505(t+2),var8506(t+2),var8507(t+2),var8508(t+2),var8509(t+2),var8510(t+2),var8511(t+2),var8512(t+2),var8513(t+2),var8514(t+2),var8515(t+2),var8516(t+2),var8517(t+2),var8518(t+2),var8519(t+2),var8520(t+2),var8521(t+2),var8522(t+2),var8523(t+2),var8524(t+2),var8525(t+2),var8526(t+2),var8527(t+2),var8528(t+2),var8529(t+2),var8530(t+2),var8531(t+2),var8532(t+2),var8533(t+2),var8534(t+2),var8535(t+2),var8536(t+2),var8537(t+2),var8538(t+2),var8539(t+2),var8540(t+2

In [6]:
# Save the DataFrame to a CSV file
reframed_data_30days_part3.to_csv('../data/reframed_data_30days_part3.csv', index=False)
reframed_data_30days_part3.shape

(9991, 87050)

In [4]:
df_test_30days_part4 = completed_data_30days[30000:40000]

In [5]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part4 = series_to_supervised(df_test_30days_part4.values, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part4.head()

--- 2.2358899116516113 seconds ---
--- 5.676138877868652 seconds ---
--- 2.4173901081085205 seconds ---
--- 8.008301496505737 seconds ---


,var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var6(t-7),var7(t-7),var8(t-7),var9(t-7),var10(t-7),var11(t-7),var12(t-7),var13(t-7),var14(t-7),var15(t-7),var16(t-7),var17(t-7),var18(t-7),var19(t-7),var20(t-7),var21(t-7),var22(t-7),var23(t-7),var24(t-7),var25(t-7),var26(t-7),var27(t-7),var28(t-7),var29(t-7),var30(t-7),var31(t-7),var32(t-7),var33(t-7),var34(t-7),var35(t-7),var36(t-7),var37(t-7),var38(t-7),var39(t-7),var40(t-7),var41(t-7),var42(t-7),var43(t-7),var44(t-7),var45(t-7),var46(t-7),var47(t-7),var48(t-7),var49(t-7),var50(t-7),var51(t-7),var52(t-7),var53(t-7),var54(t-7),var55(t-7),var56(t-7),var57(t-7),var58(t-7),var59(t-7),var60(t-7),var61(t-7),var62(t-7),var63(t-7),var64(t-7),var65(t-7),var66(t-7),var67(t-7),var68(t-7),var69(t-7),var70(t-7),var71(t-7),var72(t-7),var73(t-7),var74(t-7),var75(t-7),var76(t-7),var77(t-7),var78(t-7),var79(t-7),var80(t-7),var81(t-7),var82(t-7),var83(t-7),var84(t-7),var85(t-7),var86(t-7),var87(t-7),var88(t-7),var89(t-7),var90(t-7),var91(t-7),var92(t-7),var93(t-7),var94(t-7),var95(t-7),var96(t-7),var97(t-7),var98(t-7),var99(t-7),var100(t-7),var101(t-7),var102(t-7),var103(t-7),var104(t-7),var105(t-7),var106(t-7),var107(t-7),var108(t-7),var109(t-7),var110(t-7),var111(t-7),var112(t-7),var113(t-7),var114(t-7),var115(t-7),var116(t-7),var117(t-7),var118(t-7),var119(t-7),var120(t-7),var121(t-7),var122(t-7),var123(t-7),var124(t-7),var125(t-7),var126(t-7),var127(t-7),var128(t-7),var129(t-7),var130(t-7),var131(t-7),var132(t-7),var133(t-7),var134(t-7),var135(t-7),var136(t-7),var137(t-7),var138(t-7),var139(t-7),var140(t-7),var141(t-7),var142(t-7),var143(t-7),var144(t-7),var145(t-7),var146(t-7),var147(t-7),var148(t-7),var149(t-7),var150(t-7),var151(t-7),var152(t-7),var153(t-7),var154(t-7),var155(t-7),var156(t-7),var157(t-7),var158(t-7),var159(t-7),var160(t-7),var161(t-7),var162(t-7),var163(t-7),var164(t-7),var165(t-7),var166(t-7),var167(t-7),var168(t-7),var169(t-7),var170(t-7),var171(t-7),var172(t-7),var173(t-7),var174(t-7),var175(t-7),var176(t-7),var177(t-7),var178(t-7),var179(t-7),var180(t-7),var181(t-7),var182(t-7),var183(t-7),var184(t-7),var185(t-7),var186(t-7),var187(t-7),var188(t-7),var189(t-7),var190(t-7),var191(t-7),var192(t-7),var193(t-7),var194(t-7),var195(t-7),var196(t-7),var197(t-7),var198(t-7),var199(t-7),var200(t-7),var201(t-7),var202(t-7),var203(t-7),var204(t-7),var205(t-7),var206(t-7),var207(t-7),var208(t-7),var209(t-7),var210(t-7),var211(t-7),var212(t-7),var213(t-7),var214(t-7),var215(t-7),var216(t-7),var217(t-7),var218(t-7),var219(t-7),var220(t-7),var221(t-7),var222(t-7),var223(t-7),var224(t-7),var225(t-7),var226(t-7),var227(t-7),var228(t-7),var229(t-7),var230(t-7),var231(t-7),var232(t-7),var233(t-7),var234(t-7),var235(t-7),var236(t-7),var237(t-7),var238(t-7),var239(t-7),var240(t-7),var241(t-7),var242(t-7),var243(t-7),var244(t-7),var245(t-7),var246(t-7),var247(t-7),var248(t-7),var249(t-7),var250(t-7),...,var8456(t+2),var8457(t+2),var8458(t+2),var8459(t+2),var8460(t+2),var8461(t+2),var8462(t+2),var8463(t+2),var8464(t+2),var8465(t+2),var8466(t+2),var8467(t+2),var8468(t+2),var8469(t+2),var8470(t+2),var8471(t+2),var8472(t+2),var8473(t+2),var8474(t+2),var8475(t+2),var8476(t+2),var8477(t+2),var8478(t+2),var8479(t+2),var8480(t+2),var8481(t+2),var8482(t+2),var8483(t+2),var8484(t+2),var8485(t+2),var8486(t+2),var8487(t+2),var8488(t+2),var8489(t+2),var8490(t+2),var8491(t+2),var8492(t+2),var8493(t+2),var8494(t+2),var8495(t+2),var8496(t+2),var8497(t+2),var8498(t+2),var8499(t+2),var8500(t+2),var8501(t+2),var8502(t+2),var8503(t+2),var8504(t+2),var8505(t+2),var8506(t+2),var8507(t+2),var8508(t+2),var8509(t+2),var8510(t+2),var8511(t+2),var8512(t+2),var8513(t+2),var8514(t+2),var8515(t+2),var8516(t+2),var8517(t+2),var8518(t+2),var8519(t+2),var8520(t+2),var8521(t+2),var8522(t+2),var8523(t+2),var8524(t+2),var8525(t+2),var8526(t+2),var8527(t+2),var8528(t+2),var8529(t+2),var8530(t+2),var8531(t+2),var8532(t+2),var8533(t+2),var8534(t+2),var8535(t+2),var8536(t+2),var8537(t+2),var8538(t+2),var8539(t+2),var8540(t+2

In [6]:
# Save the DataFrame to a CSV file
reframed_data_30days_part4.to_csv('../data/reframed_data_30days_part4.csv', index=False)
reframed_data_30days_part4.shape

(9991, 87050)

In [4]:
df_test_30days_part5 = completed_data_30days[40000:]

In [5]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part5 = series_to_supervised(df_test_30days_part5.values, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part5.head()

--- 2.1863481998443604 seconds ---
--- 5.526118516921997 seconds ---
--- 2.3523306846618652 seconds ---
--- 7.802771091461182 seconds ---


,var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var6(t-7),var7(t-7),var8(t-7),var9(t-7),var10(t-7),var11(t-7),var12(t-7),var13(t-7),var14(t-7),var15(t-7),var16(t-7),var17(t-7),var18(t-7),var19(t-7),var20(t-7),var21(t-7),var22(t-7),var23(t-7),var24(t-7),var25(t-7),var26(t-7),var27(t-7),var28(t-7),var29(t-7),var30(t-7),var31(t-7),var32(t-7),var33(t-7),var34(t-7),var35(t-7),var36(t-7),var37(t-7),var38(t-7),var39(t-7),var40(t-7),var41(t-7),var42(t-7),var43(t-7),var44(t-7),var45(t-7),var46(t-7),var47(t-7),var48(t-7),var49(t-7),var50(t-7),var51(t-7),var52(t-7),var53(t-7),var54(t-7),var55(t-7),var56(t-7),var57(t-7),var58(t-7),var59(t-7),var60(t-7),var61(t-7),var62(t-7),var63(t-7),var64(t-7),var65(t-7),var66(t-7),var67(t-7),var68(t-7),var69(t-7),var70(t-7),var71(t-7),var72(t-7),var73(t-7),var74(t-7),var75(t-7),var76(t-7),var77(t-7),var78(t-7),var79(t-7),var80(t-7),var81(t-7),var82(t-7),var83(t-7),var84(t-7),var85(t-7),var86(t-7),var87(t-7),var88(t-7),var89(t-7),var90(t-7),var91(t-7),var92(t-7),var93(t-7),var94(t-7),var95(t-7),var96(t-7),var97(t-7),var98(t-7),var99(t-7),var100(t-7),var101(t-7),var102(t-7),var103(t-7),var104(t-7),var105(t-7),var106(t-7),var107(t-7),var108(t-7),var109(t-7),var110(t-7),var111(t-7),var112(t-7),var113(t-7),var114(t-7),var115(t-7),var116(t-7),var117(t-7),var118(t-7),var119(t-7),var120(t-7),var121(t-7),var122(t-7),var123(t-7),var124(t-7),var125(t-7),var126(t-7),var127(t-7),var128(t-7),var129(t-7),var130(t-7),var131(t-7),var132(t-7),var133(t-7),var134(t-7),var135(t-7),var136(t-7),var137(t-7),var138(t-7),var139(t-7),var140(t-7),var141(t-7),var142(t-7),var143(t-7),var144(t-7),var145(t-7),var146(t-7),var147(t-7),var148(t-7),var149(t-7),var150(t-7),var151(t-7),var152(t-7),var153(t-7),var154(t-7),var155(t-7),var156(t-7),var157(t-7),var158(t-7),var159(t-7),var160(t-7),var161(t-7),var162(t-7),var163(t-7),var164(t-7),var165(t-7),var166(t-7),var167(t-7),var168(t-7),var169(t-7),var170(t-7),var171(t-7),var172(t-7),var173(t-7),var174(t-7),var175(t-7),var176(t-7),var177(t-7),var178(t-7),var179(t-7),var180(t-7),var181(t-7),var182(t-7),var183(t-7),var184(t-7),var185(t-7),var186(t-7),var187(t-7),var188(t-7),var189(t-7),var190(t-7),var191(t-7),var192(t-7),var193(t-7),var194(t-7),var195(t-7),var196(t-7),var197(t-7),var198(t-7),var199(t-7),var200(t-7),var201(t-7),var202(t-7),var203(t-7),var204(t-7),var205(t-7),var206(t-7),var207(t-7),var208(t-7),var209(t-7),var210(t-7),var211(t-7),var212(t-7),var213(t-7),var214(t-7),var215(t-7),var216(t-7),var217(t-7),var218(t-7),var219(t-7),var220(t-7),var221(t-7),var222(t-7),var223(t-7),var224(t-7),var225(t-7),var226(t-7),var227(t-7),var228(t-7),var229(t-7),var230(t-7),var231(t-7),var232(t-7),var233(t-7),var234(t-7),var235(t-7),var236(t-7),var237(t-7),var238(t-7),var239(t-7),var240(t-7),var241(t-7),var242(t-7),var243(t-7),var244(t-7),var245(t-7),var246(t-7),var247(t-7),var248(t-7),var249(t-7),var250(t-7),...,var8456(t+2),var8457(t+2),var8458(t+2),var8459(t+2),var8460(t+2),var8461(t+2),var8462(t+2),var8463(t+2),var8464(t+2),var8465(t+2),var8466(t+2),var8467(t+2),var8468(t+2),var8469(t+2),var8470(t+2),var8471(t+2),var8472(t+2),var8473(t+2),var8474(t+2),var8475(t+2),var8476(t+2),var8477(t+2),var8478(t+2),var8479(t+2),var8480(t+2),var8481(t+2),var8482(t+2),var8483(t+2),var8484(t+2),var8485(t+2),var8486(t+2),var8487(t+2),var8488(t+2),var8489(t+2),var8490(t+2),var8491(t+2),var8492(t+2),var8493(t+2),var8494(t+2),var8495(t+2),var8496(t+2),var8497(t+2),var8498(t+2),var8499(t+2),var8500(t+2),var8501(t+2),var8502(t+2),var8503(t+2),var8504(t+2),var8505(t+2),var8506(t+2),var8507(t+2),var8508(t+2),var8509(t+2),var8510(t+2),var8511(t+2),var8512(t+2),var8513(t+2),var8514(t+2),var8515(t+2),var8516(t+2),var8517(t+2),var8518(t+2),var8519(t+2),var8520(t+2),var8521(t+2),var8522(t+2),var8523(t+2),var8524(t+2),var8525(t+2),var8526(t+2),var8527(t+2),var8528(t+2),var8529(t+2),var8530(t+2),var8531(t+2),var8532(t+2),var8533(t+2),var8534(t+2),var8535(t+2),var8536(t+2),var8537(t+2),var8538(t+2),var8539(t+2),var8540(t+2

In [6]:
# Save the DataFrame to a CSV file
reframed_data_30days_part5.to_csv('../data/reframed_data_30days_part5.csv', index=False)
reframed_data_30days_part5.shape

(9731, 87050)

---

### Read the datasets

In [ ]:
reframed_data_30days_part1 = pd.read_csv('../data/reframed_data_30days_part1.csv')
reframed_data_30days_part1.shape

In [2]:
# Read the datasets
reframed_data_30days_part1 = pd.read_csv('../data/reframed_data_30days_part1.csv')
reframed_data_30days_part2 = pd.read_csv('../data/reframed_data_30days_part2.csv')
reframed_data_30days_part3 = pd.read_csv('../data/reframed_data_30days_part3.csv')
reframed_data_30days_part4 = pd.read_csv('../data/reframed_data_30days_part4.csv')
reframed_data_30days_part5 = pd.read_csv('../data/reframed_data_30days_part5.csv')
# Print their shapes
reframed_data_30days_part1.shape
reframed_data_30days_part2.shape
reframed_data_30days_part3.shape
reframed_data_30days_part4.shape
reframed_data_30days_part5.shape

(9731, 87050)

In [3]:
# Concat the DataFrames
merged_data_30days = pd.concat([reframed_data_30days_part1, reframed_data_30days_part2, 
                                reframed_data_30days_part3, reframed_data_30days_part4, 
                                reframed_data_30days_part5], axis=0, ignore_index=True)
merged_data_30days.shape

(49695, 87050)

In [4]:
# Save the DataFrame to a CSV file
merged_data_30days.to_csv('../data/merged_data_30days.csv', index=False)
merged_data_30days.shape

(49695, 87050)